In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Aug  6 13:12:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount("/content/gdrive/", force_remount=True)

Mounted at /content/gdrive/


In [4]:
!pip install -U git+https://github.com/Adapter-Hub/adapter-transformers.git
#!pip install transformers==3.0.2
!pip3 install -r /content/gdrive/MyDrive/master_hpi/NLP_Project/requirements.txt

  Cloning https://github.com/Adapter-Hub/adapter-transformers.git to /tmp/pip-req-build-fxcykpki
  Running command git clone -q https://github.com/Adapter-Hub/adapter-transformers.git /tmp/pip-req-build-fxcykpki
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 895 kB 7.4 MB/s 
     |████████████████████████████████| 43 kB 2.3 MB/s 
     |████████████████████████████████| 3.3 MB 38.8 MB/s 
  Created wheel for adapter-transformers: filename=adapter_transformers-2.1.2-py3-none-any.whl size=2539220 sha256=55fc6da0c802b600d9fe7db654207397437ae4a6df03b7e65a577c97b174c97d
  Stored in directory: /tmp/pip-ephem-wheel-cache-tytjn5j8/wheels/60/0e/d5/86e64fef3085f149b504f48af9ca6e8f6da427f42da9b4bbe5
Successfully built adapter-transformers
     |████████████████████████████████| 128 kB 7.6 MB/s 
     |████████████████████████████████| 6.1 MB 18.8 MB/s 
     |████████████████████████

In [5]:
import sys
from collections import defaultdict 
from collections import defaultdict 
from transformers import BertForMaskedLM, BertModel, BertTokenizer, BertForSequenceClassification

sys.path.append('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/')
path = "/content/gdrive/MyDrive/master_hpi/NLP_Project/code/"

[autoreload of six failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
AttributeError: 'NoneType' object has no attribute 'cStringIO'
]


In [6]:
#adapter_list = ["sst"]
#for adapter in adapter_list:
#  print("!python3 evaluation.py --no-cuda False --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/ -- skip_intrasentence False --skip_intersentence False --batch_size 1")

## Sentiment Bert


In [7]:
import os
try:
    os.makedirs('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sentiment_bert/')
except OSError as e:
        pass

In [8]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/evaluation.py --load-path /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/sentiment_bert/SentimentBert.pth --input-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sentiment_bert/ --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sentiment_bert/predictions.json
#!python3 evaluation.py --no-cuda False --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/ -- skip_intrasentence False --skip_intersentence False --batch_size 1

usage: evaluation.py [-h] [--no-cuda] [--input-file INPUT_FILE]
                     [--output-dir OUTPUT_DIR] [--output-file OUTPUT_FILE]
                     [--skip-intrasentence]
                     [--load-path_model LOAD_PATH_MODEL]
                     [--load-path_adapter LOAD_PATH_ADAPTER]
                     [--skip-intersentence] [--batch-size BATCH_SIZE]
                     [--max-seq-length MAX_SEQ_LENGTH]
evaluation.py: error: ambiguous option: --load-path could match --load-path_model, --load-path_adapter


In [9]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/score_evaluation.py --gold-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --predictions-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sentiment_bert/predictions.json --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sentiment_bert/summary.json

intrasentence
	gender
		Count: 765.0
		LM Score: 36.45418751940491
		SS Score: 42.02009600705252
		ICAT Score: 30.636169188489802
	profession
		Count: 2430.0
		LM Score: 45.61401894636235
		SS Score: 45.28042588097158
		ICAT Score: 41.30844408067987
	race
		Count: 2886.0
		LM Score: 49.101373831914486
		SS Score: 70.14426379380126
		ICAT Score: 29.319153289751775
	religion
		Count: 237.0
		LM Score: 44.7816091954023
		SS Score: 50.62068965517241
		ICAT Score: 44.225699564011094
	overall
		Count: 2106.0
		LM Score: 46.01211049519047
		SS Score: 56.40087724103842
		ICAT Score: 40.121753077574276
intersentence
	gender
		Count: 726.0
		LM Score: 85.78311081028474
		SS Score: 58.763293871989525
		ICAT Score: 70.74825862460541
	profession
		Count: 2481.0
		LM Score: 80.69744136512767
		SS Score: 65.2029215734073
		ICAT Score: 56.160703920154276
	race
		Count: 2928.0
		LM Score: 84.90385064737741
		SS Score: 70.48116164220478
		ICAT Score: 50.12526086428642
	religion
		Count: 234.0
		LM Score

## SST Adapter


In [10]:
import os
try:
    os.makedirs('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst/')
except OSError as e:
        pass

In [11]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/evaluation.py --load-path_model /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/sst/pytorch_model.bin --load-path_adapter /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/sst/pytorch_adapter.bin --input-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst/ --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst/predictions.json
#!python3 evaluation.py --no-cuda False --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/ -- skip_intrasentence False --skip_intersentence False --batch_size 1

Loading /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json...
Downloading: 100% 232k/232k [00:00<00:00, 1.70MB/s]
Downloading: 100% 28.0/28.0 [00:00<00:00, 24.2kB/s]
Downloading: 100% 466k/466k [00:00<00:00, 3.27MB/s]
---------------------------------------------------------------
                     ARGUMENTS                 
Pretrained Model: /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/sst/pytorch_model.bin
Skip Intrasentence: False
Skip Intersentence: False
Batch Size: 1
Max Seq Length: None
CUDA: True
---------------------------------------------------------------

Evaluating bias on intersentence tasks...
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2160: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to tru

In [12]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/score_evaluation.py --gold-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --predictions-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst/predictions.json --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst/summary.json

intrasentence
	gender
		Count: 765.0
		LM Score: 45.91933364542059
		SS Score: 51.118456905413424
		ICAT Score: 44.89215772926652
	profession
		Count: 2430.0
		LM Score: 39.872843116209225
		SS Score: 48.60059298275029
		ICAT Score: 38.75687638711882
	race
		Count: 2886.0
		LM Score: 34.97359197774884
		SS Score: 52.864209424855176
		ICAT Score: 32.97015814247469
	religion
		Count: 237.0
		LM Score: 39.28735632183908
		SS Score: 57.241379310344826
		ICAT Score: 33.597463337296865
	overall
		Count: 2106.0
		LM Score: 38.38341784942986
		SS Score: 51.19035488306916
		ICAT Score: 37.46962007211081
intersentence
	gender
		Count: 726.0
		LM Score: 70.6322994638212
		SS Score: 43.113995505299854
		ICAT Score: 60.904812832243614
	profession
		Count: 2481.0
		LM Score: 67.41529759121529
		SS Score: 49.67834286314079
		ICAT Score: 66.98160535914126
	race
		Count: 2928.0
		LM Score: 67.1786963697188
		SS Score: 45.521771296103886
		ICAT Score: 61.16186504225488
	religion
		Count: 234.0
		LM Scor

# RoBERTa


In [26]:
import os
try:
    os.makedirs('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/roberta/')
except OSError as e:
        pass

In [27]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/evaluation.py --load-path_model /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/roberta/pytorch_model_head.bin --load-path_adapter /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/roberta/pytorch_adapter.bin --input-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/roberta/ --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/roberta/predictions.json


Loading /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json...
---------------------------------------------------------------
                     ARGUMENTS                 
Pretrained Model: /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/roberta/pytorch_model_head.bin
Skip Intrasentence: False
Skip Intersentence: False
Batch Size: 1
Max Seq Length: None
CUDA: True
---------------------------------------------------------------

Evaluating bias on intersentence tasks...
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2160: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selecte

In [28]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/score_evaluation.py --gold-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --predictions-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/roberta/predictions.json --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/roberta/summary.json

intrasentence
	gender
		Count: 765.0
		LM Score: 57.61251872121437
		SS Score: 48.907944068813634
		ICAT Score: 56.35419686561262
	profession
		Count: 2430.0
		LM Score: 56.75200938428044
		SS Score: 49.984738677571286
		ICAT Score: 56.73468717000662
	race
		Count: 2886.0
		LM Score: 65.65826879356138
		SS Score: 52.66689783697006
		ICAT Score: 62.15619089306643
	religion
		Count: 237.0
		LM Score: 68.22988505747126
		SS Score: 57.60919540229886
		ICAT Score: 57.846394503897464
	overall
		Count: 2106.0
		LM Score: 61.35535190482505
		SS Score: 51.36022163738093
		ICAT Score: 59.68621436022376
intersentence
	gender
		Count: 726.0
		LM Score: 10.483877032790076
		SS Score: 44.748035780644464
		ICAT Score: 9.382658091663341
	profession
		Count: 2481.0
		LM Score: 16.047803449314387
		SS Score: 47.96422551891591
		ICAT Score: 15.394409274523037
	race
		Count: 2928.0
		LM Score: 16.186329153870247
		SS Score: 56.366103567237545
		ICAT Score: 14.125452198531557
	religion
		Count: 234.0
		LM 

#Language Poem (pre-trained)


In [16]:
import os
try:
    os.makedirs('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/lmpoem/')
except OSError as e:
        pass

In [17]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/evaluation.py --load-path_model /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/lmpoem/pytorch_adapter.bin --load-path_adapter /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/lmpoem/pytorch_adapter.bin --input-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/lmpoem/ --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/lmpoem/predictions.json
#!python3 evaluation.py --no-cuda False --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/ -- skip_intrasentence False --skip_intersentence False --batch_size 1

Loading /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json...
---------------------------------------------------------------
                     ARGUMENTS                 
Pretrained Model: /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/lmpoem/pytorch_adapter.bin
Skip Intrasentence: False
Skip Intersentence: False
Batch Size: 1
Max Seq Length: None
CUDA: True
---------------------------------------------------------------

Evaluating bias on intersentence tasks...
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2160: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected am

In [18]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/score_evaluation.py --gold-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --predictions-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/lmpoem/predictions.json --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/lmpoem/summary.json

intrasentence
	gender
		Count: 765.0
		LM Score: 60.60657152831065
		SS Score: 52.33507876986139
		ICAT Score: 57.776149158513775
	profession
		Count: 2430.0
		LM Score: 57.82664639069813
		SS Score: 48.80567859483745
		ICAT Score: 56.445374359234606
	race
		Count: 2886.0
		LM Score: 49.86421728632914
		SS Score: 57.931965700435626
		ICAT Score: 41.9537920624445
	religion
		Count: 237.0
		LM Score: 47.793103448275865
		SS Score: 53.839080459770116
		ICAT Score: 44.12347205707491
	overall
		Count: 2106.0
		LM Score: 54.16906632476871
		SS Score: 53.602394330870005
		ICAT Score: 50.266299576031344
intersentence
	gender
		Count: 726.0
		LM Score: 89.73087098087099
		SS Score: 55.53595760117499
		ICAT Score: 79.79594503553889
	profession
		Count: 2481.0
		LM Score: 79.9801205002169
		SS Score: 55.0862559473238
		ICAT Score: 71.84413322897885
	race
		Count: 2928.0
		LM Score: 81.97717909879573
		SS Score: 46.90784723328599
		ICAT Score: 76.90745987564071
	religion
		Count: 234.0
		LM Score:

#Common Sense Winogrande (pre-trained)


In [19]:
import os
try:
    os.makedirs('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/winogrande/')
except OSError as e:
        pass

In [20]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/evaluation.py --load-path_model /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/winogrande/pytorch_adapter.bin --load-path_adapter /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/winogrande/pytorch_adapter.bin --input-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/winogrande/ --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/winogrande/predictions.json


Loading /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json...
---------------------------------------------------------------
                     ARGUMENTS                 
Pretrained Model: /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/winogrande/pytorch_adapter.bin
Skip Intrasentence: False
Skip Intersentence: False
Batch Size: 1
Max Seq Length: None
CUDA: True
---------------------------------------------------------------

Evaluating bias on intersentence tasks...
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2160: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selecte

In [21]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/score_evaluation.py --gold-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --predictions-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/winogrande/predictions.json --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/winogrande/summary.json

intrasentence
	gender
		Count: 765.0
		LM Score: 68.57429510907772
		SS Score: 54.58370791849053
		ICAT Score: 62.287804319150005
	profession
		Count: 2430.0
		LM Score: 69.34632676916495
		SS Score: 48.82774310440195
		ICAT Score: 67.72049257437395
	race
		Count: 2886.0
		LM Score: 69.79387709212011
		SS Score: 45.53308433875475
		ICAT Score: 63.55860983928376
	religion
		Count: 237.0
		LM Score: 61.17241379310345
		SS Score: 43.3103448275862
		ICAT Score: 52.98796670630202
	overall
		Count: 2106.0
		LM Score: 69.1421464665995
		SS Score: 47.84546130373283
		ICAT Score: 66.16275786449428
intersentence
	gender
		Count: 726.0
		LM Score: 16.173908619560795
		SS Score: 43.841110580241015
		ICAT Score: 14.181642326097563
	profession
		Count: 2481.0
		LM Score: 20.907715881325444
		SS Score: 40.66319326109833
		ICAT Score: 17.003489830609425
	race
		Count: 2928.0
		LM Score: 17.98837111982118
		SS Score: 41.664682068093256
		ICAT Score: 14.989595272604404
	religion
		Count: 234.0
		LM Scor

#SST Pretrained


In [22]:
import os
try:
    os.makedirs('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst-pretrained/')
except OSError as e:
        pass

In [23]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/evaluation.py --load-path_model /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/sst_pretrained/pytorch_adapter.bin --load-path_adapter /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/sst_pretrained/pytorch_adapter.bin --input-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst-pretrained/ --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst-pretrained/predictions.json


Loading /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json...
---------------------------------------------------------------
                     ARGUMENTS                 
Pretrained Model: /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/sst_pretrained/pytorch_adapter.bin
Skip Intrasentence: False
Skip Intersentence: False
Batch Size: 1
Max Seq Length: None
CUDA: True
---------------------------------------------------------------

Evaluating bias on intersentence tasks...
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2160: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy sel

In [24]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/score_evaluation.py --gold-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --predictions-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst-pretrained/predictions.json --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst-pretrained/summary.json

intrasentence
	gender
		Count: 765.0
		LM Score: 51.475735053995926
		SS Score: 60.03565452261104
		ICAT Score: 41.14388118800869
	profession
		Count: 2430.0
		LM Score: 49.40798058808308
		SS Score: 47.84946663843627
		ICAT Score: 47.28291037643976
	race
		Count: 2886.0
		LM Score: 43.56339419514788
		SS Score: 46.37599926220602
		ICAT Score: 40.40591874106737
	religion
		Count: 237.0
		LM Score: 45.793103448275865
		SS Score: 45.241379310344826
		ICAT Score: 41.43486325802616
	overall
		Count: 2106.0
		LM Score: 46.869092019653195
		SS Score: 48.621527287970245
		ICAT Score: 45.576936731919126
intersentence
	gender
		Count: 726.0
		LM Score: 89.69184096901489
		SS Score: 59.3633097980924
		ICAT Score: 72.89559110193244
	profession
		Count: 2481.0
		LM Score: 86.00529644211514
		SS Score: 61.364194314453876
		ICAT Score: 66.45767842530704
	race
		Count: 2928.0
		LM Score: 88.7734208442171
		SS Score: 60.473608971924996
		ICAT Score: 70.17785890376778
	religion
		Count: 234.0
		LM Scor